In [ ]:
%load_ext autoreload
%autoreload 2
from keras.applications.vgg16 import VGG16

import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import keras

from sklearn.utils import class_weight

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import models.finalModels as finalModels
import keras.applications as kap
from imgaug import augmenters as iaa

import h5py
from imgaug import augmenters as iaa
from keras.utils import np_utils

import pickle

import utils
import preprocessing as pp
pp.init()

allTrainingFolders = [x for x in os.listdir(pp.trainPath) if x.startswith('n')]
np.random.shuffle(allTrainingFolders)
num = 25
trainingFolders = allTrainingFolders[0:num]
  


In [ ]:
# pp.trainPath = 'E:\\SCHOOL CMPS 240\\ILSVRC\Data\\CLS-LOC\\train'

# allTrainingFolders = [x for x in os.listdir(pp.trainPath) if x.startswith('n')]
# np.random.shuffle(allTrainingFolders)
# num = 25
# trainingFolders = allTrainingFolders[0:num]

# trainingFolders

In [ ]:
# with open (os.path.join('output', 'trainingFolderOrder25.text'), 'rb') as fp:     
#     trainingFolders = pickle.load(fp)

In [ ]:
parse =False
train = True
saveModel = False
includeAugmented = False
onlyAugmented = False

augments = [iaa.Rot90(1), iaa.Rot90(2), iaa.Fliplr(1), iaa.Flipud(1), iaa.Noop(), iaa.Dropout((0.01, 0.1), per_channel=0.5), 
            iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), iaa.PerspectiveTransform(scale=(0.01, 0.1))]
# augments = []
h5filename = '{0}-finalclasses.h5'.format(num)
h5file = os.path.join(pp.h5Path, h5filename)

if parse:
    if os.path.isfile(os.path.join(pp.h5Path, h5filename)):
        os.remove(os.path.join(pp.h5Path, h5filename))
    pp.parseImages(trainingFolders, h5filename)
#     pp.shuffleH5(h5file)
    #write to text file
    with open(os.path.join('output', 'trainingFolderOrder25.text'), 'wb') as fp:     
        pickle.dump(trainingFolders, fp)
    
#read back    
with open (os.path.join('output', 'trainingFolderOrder25.text'), 'rb') as fp:     
    trainingFolders = pickle.load(fp)
    
encoder = LabelBinarizer()
##MAKE SURE classLabels is set to ALL the folders you will train on, even if doing in batches
encoder = encoder.fit(trainingFolders)

h5db = h5py.File(h5file, 'r')

x_val = h5db['x_val'][:]
y_val = h5db['y_val'][:]

y_val = [k.decode('utf-8') for k in y_val]

y_val = encoder.transform(y_val)
val_data = (np.array(x_val), np.array(y_val))

h5db.close()
    
#NOTE: KEEP BATCH SIZE = # of all folders for now because we need to shuffle H5. 

filepath=os.path.join(pp.outputModelPath, "best-epoch-finalmodel-{epoch:02d}-{val_acc:.2f}.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                   patience=20, min_delta=0.5)
# tb = keras.callbacks.TensorBoard(log_dir='./tensor-graph', histogram_freq=0,             
#                                  write_graph=False, write_images=False)

callbacks = [checkpoint]
epochs = 100
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# adam = keras.optimizers.Adam(lr=0.01, decay=1e-6)
model = finalModels.finalModel(len(encoder.classes_)) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = load_model((os.path.join(pp.outputModelPath, 'best-epoch-finalModel-37-0.68.hdf5')))

y_ints = [y.argmax() for y in y_val] 
class_weights = class_weight.compute_class_weight('balanced',                                                  
                                                np.unique(y_ints),                                                  
                                                y_ints)

batch_size = 20

##FIT MODEL
if train:
    train_generator = pp.DataGenerator3(h5file, trainingFolders, batch_size=batch_size, augmentations = augments, augmentReplace = False
                                       , augment_pct = 0.25, shuffle=True)
    val_generator = pp.DataGenerator3(h5file, trainingFolders, batch_size=batch_size, isValidation = True)
    print(len(train_generator))
    results = model.fit_generator(train_generator, validation_data = val_data, epochs=epochs, callbacks = callbacks, verbose=1,
                              use_multiprocessing=True, workers=4, class_weight = class_weights)
    with open(os.path.join(pp.trainhistory, 'finalmodelhistory.pkl'), 'wb') as file_pi:         
        pickle.dump(results.history, file_pi)

    if saveModel:
        count = 1
        outputName = ""
        while True:
            k = ''
            if includeAugmented:
                k = 'with-aug'
            if onlyAugmented:
                k = 'aug-only'
            s = pp.baseModelName.format(epoch, count, k) + ".h5" 
            if not s in os.listdir(pp.outputModelPath):         
                outputName = s         
                break     
            else:         
                count+=1 
        print(outputName)
        model.save(os.path.join(pp.outputModelPath, outputName))
        


In [ ]:
with open(os.path.join(pp.trainhistory, 'finalmodelhistory.pkl'), 'wb') as file_pi:         
        pickle.dump(results.history, file_pi)

In [ ]:
with open (os.path.join(pp.trainhistory, 'finalmodelhistory.pkl'), 'rb') as fp:     
    results = pickle.load(fp)
    print(results['val_acc'])

In [ ]:
import preprocessing as pp
import os
import h5py

h5filename = '25-finalclasses.h5'
h5file = os.path.join(pp.h5Path, h5filename)
h5db = h5py.File(h5file, 'r') 
print(len(h5db['x_train']))
h5db.close()


In [ ]:
# model1 = load_model((os.path.join(pp.outputModelPath, 'best-epoch-Res25-100-0.63.hdf5')))
model1 = load_model((os.path.join(pp.outputModelPath, 'best-epoch-RESNET50-37-0.53.hdf5')))
model2 = load_model((os.path.join(pp.outputModelPath, 'best-epoch-SSD7-30-0.61.hdf5')))
model3 = load_model((os.path.join(pp.outputModelPath, 'best-epoch-finalmodel-99-0.66.hdf5')))

# model.history
h5filename = '{0}-finalclasses.h5'.format(num)
h5file = os.path.join(pp.h5Path, h5filename)

h5db = h5py.File(h5file, 'r')
print("ResNet50 top 5: ", pp.top5accuracies(model1, h5db['x_val'][:], h5db['y_val'][:], encoder))
print("SSD7 top 5: ", pp.top5accuracies(model2, h5db['x_val'][:], h5db['y_val'][:], encoder))
print("Final Model top 5: ", pp.top5accuracies(model3, h5db['x_val'][:], h5db['y_val'][:], encoder))
# # df = pp.predictionsToDataframe(model, h5db['x_val'][:], h5db['y_val'][:], encoder)
# # df[700:1100]
h5db.close()


                              

In [ ]:
resNet = None
ssd = None
FinalNet = None
FinalNetaugs = None
with open (os.path.join(pp.trainhistory, 'Reshistory.pkl'), 'rb') as fp:     
    resNet = pickle.load(fp)
with open (os.path.join(pp.trainhistory, 'SSDhistory.pkl'), 'rb') as fp2:     
     ssd = pickle.load(fp2)
with open (os.path.join(pp.trainhistory, 'finalmodelhistory.pkl'), 'rb') as fp3:     
     FinalNet = pickle.load(fp3)
with open (os.path.join(pp.trainhistory, 'finalmodelAUGShistory.pkl'), 'rb') as fp4:     
     FinalNetaugs = pickle.load(fp4)
    

# Plot training & validation accuracy values
from matplotlib.ticker import FormatStrFormatter

epochs = 100

x = np.arange(epochs)+1
plt.plot(x, resNet['acc'], color = 'red')
plt.plot(x, resNet['val_acc'], color = 'orange')
# plt.xticks(x)
# plt.xticks([10*i for i in range(0, 11)])
plt.title('ResNet50 Accuracy')
plt.ylabel('Accuracy')
plt.yticks([0.1*i for i in range(0, 11)])
# plt.ylim(0.5, 1.0)
plt.xlabel('Epoch')
plt.legend(['Train - ResNet', 'Val - ResNet'], loc='upper left')
# plt.savefig(os.path.join('output', 'figs', outputName+'-accuracy.png'))
plt.show()

x = np.arange(epochs)+1
plt.plot(x, ssd['acc'], color = 'brown')
plt.plot(x, ssd['val_acc'], color = 'green')
# plt.xticks(x)
# plt.xticks([10*i for i in range(0, 11)])
plt.title('SSD7 Accuracy')
plt.ylabel('Accuracy')
plt.yticks([0.1*i for i in range(0, 11)])
# plt.ylim(0.5, 1.0)
plt.xlabel('Epoch')
plt.legend(['Train - SSD', 'Val - SSD'], loc='upper left')
# plt.savefig(os.path.join('output', 'figs', outputName+'-accuracy.png'))
plt.show()

x = np.arange(epochs)+1
plt.plot(x, FinalNet['acc'], color = 'blue')
plt.plot(x, FinalNet['val_acc'], color = 'purple')
# plt.xticks(x)
# plt.xticks([10*i for i in range(0, 11)])
plt.yticks([0.1*i for i in range(0, 11)])
plt.title('Final Model Accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.5, 1.0)
plt.xlabel('Epoch')
plt.legend(['Train - FinalNet', 'Val - FinalNet'], loc='upper left')
# plt.savefig(os.path.join('output', 'figs', outputName+'-accuracy.png'))
plt.show()

x = np.arange(epochs)+1
plt.plot(x, FinalNetaugs['acc'], color = 'black')
plt.plot(x, FinalNetaugs['val_acc'], color = 'gray')
# plt.xticks(x)
# plt.xticks([10*i for i in range(0, 11)])
plt.yticks([0.1*i for i in range(0, 11)])
plt.title('Final Model w/ Augs Accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.5, 1.0)
plt.xlabel('Epoch')
plt.legend(['Train - FinalNetAugs', 'Val - FinalNetAugs'], loc='upper left')
# plt.savefig(os.path.join('output', 'figs', outputName+'-accuracy.png'))
plt.show()

# # Plot training & validation loss values
# plt.plot(x, resNet['loss'])
# plt.plot(x, resNet['val_loss'])
# plt.plot(x, ssd['loss'])
# plt.plot(x, ssd['val_loss'])
# plt.plot(x, SwagNet['loss'])
# plt.plot(x, SwagNet['val_loss'])
# # plt.xticks(x)
# # plt.xticks([10*i for i in range(0, 11)])
# plt.title('Model loss')
# plt.ylabel('Loss')
# # plt.ylim(0, 0.7)
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# # plt.savefig(os.path.join('output', 'figs', outputName+'-loss.png'))
# plt.show()